In [5]:
''' Script to parse R1/R2/RI files into R4 files new version written using Biopython and Python 3'''
'''Input: R1,R2,RI file
ouutput: R4A and R4B files'''
''' the output file (R4A/B or R4 for single end reads) contains all reads that pass barcode filter in fastq format, 
where the name of each read follows this format: barcode-number where barcode is the barcode assigned to the read and number is an integer id 
assigned to each read'''

Qscore = dict((chr(i),i-33) for i in range(33,90)) #converts Qscore character to a number Qscore

#R1 = read 1 file
#RI = I7 read file
#R2 = read 2 file
#outfile A = read 1 output file
#outfile B = read 2 output file
#barversion is the version of the barcode used, which determines how the barcodes are parsed from the sequence
#for DoTA-seq v3, use barversion = 4
#for DoTA-seq v1,2 use barversion = 3 
#for the BFragilis dataset, use barversion = 1 (but all new libraries should use barversion = 4)
#bar cutoff is the qscore cutoff to remove barcodes which dont have at least 70% of bases above this cutoff
#paired - the dataset is paired end reads or not

def mapit (R1, RI, R2, outfileA, outfileB, barversion, barcutoff = 20, limit = 0, paired = True):

    #barcutoff is the quality score at which we cut off if 70% of base in the read is below that score
    num = 0
    filtered = 0        #number of reads that failed filter
    passed = 0          #number of reads that passed filter
    desynch = 0
    count = 0
    frameshift = 0
    
    try:
        for read in RI:     #this is for getting the barcode
            num += 1
            passfilter = True #resets pass filter for determining if we should skip this read

            if limit != 0: #stop the cycle at limit if a limit is given
                if num > limit:
                    break

            ID = read.id #just the clusterID
            
            if barversion == 4: #if there's definitely no frameshift, just select the correct positions
                bar = read.seq[2:17]
                barscore = read.letter_annotations["phred_quality"][2:17] #adds the quality scores lines                
                
            elif barversion == 3: #the 20base barcode read with ver2 barcode, works with ver1 barcode as well
                #barcode looks like this:
                #GA-N15-CCAG and various frameshifts of it
                
                #first I'll try lookin for the CC starting from the end, and take 15bp backwards from it
                bar = read.seq   #next line is the sequence, we extract barcode
                barscore = read.letter_annotations["phred_quality"] #adds the quality scores lines                
                CC = bar.rfind("CC")
                
                if CC >=15:
                    bar = bar[:CC] #set the end to where CC was found
                    barscore = barscore[:CC]
                    bar = bar[-15:] #count 15 backwards from that
                    barscore = barscore[-15:]
                        
            elif barversion == 2: #(barcode V2 with 18 bases of read)
                #Barcode looks like this:
                # GA-N(15)-CC, with various frameshifts in it
                #identify barcode: 
                #frameshift 1: XA-N15
                #frameshfit 2: A-N15-C
                #frameshfit 3: N15-CC
                #frameshfit 4: XXA-N14 - we should sequence longer to get the full bar sequence

                bar = read.seq   #next line is the sequence, we extract barcode
                barscore = read.letter_annotations["phred_quality"] #adds the quality scores lines
            
                #This method removes ~50% of barcodes
                #identify the frameshift 1, which should end cleanly in the barcode 
                if (bar[1] == "A" and bar[-2:] != "CC"):
                    bar = bar[2:]
                    barscore = barscore[2:]
                #frameshift 2, which should end in a C
                elif (bar[0] == "A" and bar[-1] == "C"):
                    bar = bar[1:-1]
                    barscore = barscore[1:-1]
                #frameshift 3, which should end in CC
                elif bar[-2:] == "CC":
                    bar = bar[:-2]
                    barscore = barscore[:-2]
              
            elif barversion == 1: #for Q5 + BarV1 where the barcode is predictable
                #Barcode looks like G-N15 so just grab the N15 segment
                bar = read.seq[1:]   #next line is the sequence, we extract barcode
                barscore = read.letter_annotations["phred_quality"][1:] #adds the quality scores lines
       
            #check barcode quality score, if fail, then flag as failed read  -> this method removes ~50% of barcodes                 
            pass_bases = 0
            for s in barscore:
                if s > barcutoff:
                    pass_bases += 1

            if len(barscore) != 15:
                frameshift += 1
                readscore = 0
            else:
                readscore = pass_bases*1.0/len(barscore)

            if readscore > 0.7:     #70% of bases pass then its a pass
                passfilter = True
                passed += 1
            else:
                passfilter = False
                filtered += 1
   
            for read in R1:
                if passfilter == False:
                    break #skip this read if no pass filter

                if read.id[21:44] == ID[21:44]:     #check we are synchronized in both files
                    #go ahead and store in the sequence data
                    read.id = f"{bar}-{num}:1" #reassign read ID to the barcode
                    outfileA.write(read.format("fastq"))                          
                    break
                else: 
                    desynch += 0.5
            
            if paired == False: #skip the bottom if no read 2
                continue 
  
            for read in R2:
                
                if passfilter == False:
                    break #skip this read if no pass filter

                if read.id[21:44] == ID[21:44]:     #check we are synchronized in both files
                    #go ahead and store in the sequence data
                    read.id = f"{bar}-{num}:2" #reassign read ID to the barcode
                    outfileB.write((read.format("fastq")))                          
                    break
                else: 
                    desynch += 0.5              
            
        print ("%d number of barcode reads looked at" % num)    
        print ("%d number of bars passed filter" % passed)
        print ("%d number of bars failed filter" % filtered)
        print ("%d number of times desynchronized" % desynch)   
        print ("%f bars filter pass rate!" % (1.0*passed/num))
        print ("%f bars failed due to frameshiftpass rate!" % (1.0*frameshift/num))
        return
    
    except (IOError):
        #print num
        print ("bad file path")
        return

In [7]:
import os
from Bio import SeqIO

os.getcwd()
os.chdir("/mnt/scratch/freeman/BF/") #Directory where raw fastQ files will be found    
files = os.listdir(os.getcwd())

#get only the fastqs
files = [f for f in files if f[-5:] == "fastq"]
print(files)

#get the library prefixes, which is used to name files later on
prefix = set([f[:-9] for f in files]) #Prefix is basically the name of the different libraries for record keeping purposes
prefix = list(prefix)
prefix.sort()
print(prefix)

['21-8-I2.fastq', '21-2-R1.fastq', '21-8-R1.fastq', '21-2-I2.fastq', '21-2-I1.fastq', '21-8-I1.fastq']
['21-2', '21-8']


In [8]:
#Process all the fastq files found
#Note that some options are not used because only single end reads are used in this dataset
import timeit
tic = timeit.default_timer()

#do R4 processing for all of them at once

for p in prefix:
    print("Processing Library {}".format(p))
    #First step is to open the files
    R1 = SeqIO.parse(f"{p}-R1.fastq", "fastq")
    RI = SeqIO.parse(f"{p}-I1.fastq", "fastq")
    R4A = open("{}-R4A.fastq".format(p), "w")
    mapit(R1 = R1,RI = RI, R2 = None, outfileA = R4A, outfileB = None, barversion = 1, barcutoff = 20, limit = 0, paired = False)    
    R4A.close()
    
toc = timeit.default_timer()
print ("%d seconds processing time" % (toc-tic))



Processing Library 21-2
3203530 number of barcode reads looked at
3093012 number of bars passed filter
110518 number of bars failed filter
0 number of times desynchronized
0.965501 bars filter pass rate!
0.000000 bars failed due to frameshiftpass rate!
Processing Library 21-8
1236273 number of barcode reads looked at
1092277 number of bars passed filter
143996 number of bars failed filter
0 number of times desynchronized
0.883524 bars filter pass rate!
0.000000 bars failed due to frameshiftpass rate!
235 seconds processing time


In [9]:
import subprocess
#map the reads to reference using Bowtie2
for p in prefix:
    os.system(f"echo bowtie2 aligning library {p} to BF")
    subprocess.run(["bowtie2", f"--very-sensitive -x ~/SICMA-analysis/BF/PS-operons -U {p}-R4A.fastq -S {p}-BF.sam --no-unal --threads 30"])
